In [1]:
%env BASEPATH=/home/jonathangehmayr/sato
# RAW_DIR can be empty if using extracted feature files.
%env RAW_DIR=$BASEPATH/tmp
%env SHERLOCKPATH=$BASEPATH/sherlock
%env EXTRACTPATH=$BASEPATH/extract
#export PYTHONPATH=$PYTHONPATH:$SHERLOCKPATH
#export PYTHONPATH=$PYTHONPATH:$BASEPATH
%env TYPENAME='atd' 

%env CUDA_VISIBLE_DEVICES=""

env: BASEPATH=/home/jonathangehmayr/sato
env: RAW_DIR=$BASEPATH/tmp
env: SHERLOCKPATH=$BASEPATH/sherlock
env: EXTRACTPATH=$BASEPATH/extract
env: TYPENAME='atd'
env: CUDA_VISIBLE_DEVICES=""


In [2]:
import pickle
import pandas as pd
from IPython.display import display
from extract.helpers.read_raw_data import get_atd_dfs

In [3]:
from gensim.models import word2vec
print(word2vec.FAST_VERSION)

-1


In [ ]:
"""
consider to relabel all root columns to always be entity

export BASEPATH=/home/jonathangehmayr/sato
# RAW_DIR can be empty if using extracted feature files.
export RAW_DIR=$BASEPATH/tmp
export SHERLOCKPATH=$BASEPATH/sherlock
export EXTRACTPATH=$BASEPATH/extract
#export PYTHONPATH=$PYTHONPATH:$SHERLOCKPATH
#export PYTHONPATH=$PYTHONPATH:$BASEPATH
export TYPENAME='atd' 

CUDA_VISIBLE_DEVICES=""

# right now using python 3.9
#python3.6 -m venv .venv
source .venv9/bin/activate


cd $BASEPATH/extract

python extract_header.py webtables0-p1

python extract_features.py webtables0-p1 --f sherlock --num_processes 4



make gensim work
install from source code 
activate venv sato 
cd gensim 
python setup.py install



We need:
 types.json file for valid types like entity target etc.

 its loading the pkl files from tmp for each separate feature group

 dataframe iter that yields the raw dfs
 header_iter is not importat to us cause all columns are important
 extract_fun needs to be adapted to not use headers as labels
    --> sherlock_features
limitedb with max col size
    
train crf_lf.py:
     first evaluates sherlock model as baseline
     try with evaluate mode
     where do the predictions be made? --> in eval_batch
          emissions = classifier(table_batch).view(batch_size, MAX_COL_COUNT, -1)
          pred = model.decode(emissions, mask_batch)
          --> CRF takes the output of sherlock and improves it.
          trains sherlock and crf at the same time 

     understood until line 240
     continue with dataset

extract_features.py:
     execute onece for column features and once for topic features with feature param
     seems to work --> done

topic_features_LDA
     added sampling to topic feature

     
split_train_test.py
     should work --> done

     
extract_header.py
     rewrite get_valid_headers(df_iter) to get correctly labeled data --> done 
     works

     
datasets.py
     line 193 we get the labels from the header file
     field_names --> labels

read_raw_data.py
     locator is just the path to the csv file --> not really important

     continue to write generator function for atd data

word_embeddings.py
     changed the preprocessing
     

See how to integrate new features like from col name

col_name_word_embedding
     --> new feature extraction function based on word embeddings feature

train_CRF_LC_atd
     --sherlock_feature_groups add name feature

models_sherlock
     adapt input_dim_dict, embedding_dim_dict

sherlock_features
     add new feature extraction

maybe sherlock needs to be trained separately
"""



In [7]:
# load pkl file
with open('/home/jonathangehmayr/sato/tmp/webtables0-p1_type78_header_valid.pkl', 'rb') as f:
    data = pickle.load(f)

In [8]:
data
"""
table_id --> the table
field_list --> the columns
field_names --> the column names
"""

,field_list,field_names
table_id,,
1438042981525.10/warc/CC-MAIN-20150728002301-00146-ip-10-236-191-2.ec2.internal.json.gz+15-Index of /results/9f3/9f3ff04caef9874fe015855759540a78cd0d9fe4/-0,"[0, 3]","[46, 75]"
1438042981525.10/warc/CC-MAIN-20150728002301-00146-ip-10-236-191-2.ec2.internal.json.gz+16-Aviation Safety Network > ASN Aviation Safety WikiBase > ASN Aviation Safety Database results-0,"[1, 3, 5]","[75, 49, 44]"
1438042981525.10/warc/CC-MAIN-20150728002301-00146-ip-10-236-191-2.ec2.internal.json.gz+22-Toronto Raptors Three Point Contest Participants - RealGM-0,"[2, 5, 6]","[73, 76, 3]"
"1438042981525.10/warc/CC-MAIN-20150728002301-00146-ip-10-236-191-2.ec2.internal.json.gz+23-Donnell Minton Player Profile, Lamar, NCAA Stats, Game Logs, Bests, Awards - RealGM-0",[2],[13]
"1438042981525.10/warc/CC-MAIN-20150728002301-00146-ip-10-236-191-2.ec2.internal.json.gz+24-Donnell Minton Player Profile, Lamar, NCAA Stats, Game Logs, Bests, Awards - RealGM-1",[2],[13]
...,...,...
1438042981525.10/warc/CC-MAIN-20150728002301-00027-ip-10-236-191-2.ec2.internal.json.gz+1842-R&R Hall of Famers by 1950s song Quiz - By buckeyebob-1,[1],[6]
1438042981525.10/warc/CC-MAIN-20150728002301-00027-ip-10-236-191-2.ec2.internal.json.gz+1843-R&R Hall of Famers by 1950s song Quiz - By buckeyebob-2,[1],[6]
1438042981525.10/warc/CC-MAIN-20150728002301-00027-ip-10-236-191-2.ec2.internal.json.gz+1844-R&R Hall of Famers by 1950s song Quiz - By buckeyebob-3,[1],[6]


In [9]:
type(data)

pandas.core.frame.DataFrame

In [10]:
"tmp/webtables1-p1_type78_sherlock_features.pkl"
# load pkl file
with open("tmp/webtables1-p1_type78_sherlock_features.pkl", 'rb') as f:
    data = pickle.load(f)

In [17]:
with pd.option_context('display.max_rows', 10, 'display.max_columns', None):
    display(data.sample(10))

Unnamed: 0 field_id  \
table_id                                                                  
1438042986022.41/warc/CC-MAIN-20150728002306-00...       40394        1   
1438042986022.41/warc/CC-MAIN-20150728002306-00...       24394        0   
1438042986022.41/warc/CC-MAIN-20150728002306-00...       16078        0   
1438042986022.41/warc/CC-MAIN-20150728002306-00...       50717        0   
1438042986022.41/warc/CC-MAIN-20150728002306-00...       11803        0   
1438042986022.41/warc/CC-MAIN-20150728002306-00...       53923        0   
1438042986022.41/warc/CC-MAIN-20150728002306-00...        7453        0   
1438042986022.41/warc/CC-MAIN-20150728002306-00...       13552        0   
1438042986022.41/warc/CC-MAIN-20150728002306-00...       52850        1   
1438042986022.41/warc/CC-MAIN-20150728002306-00...       27942        0   

                                                          header  \
table_id                                                           
1438042986022.41/warc/CC-MAIN-20150728002306-00...  Manufacturer   
1438042986022.41/warc/CC-MAIN-20150728002306-00...          Year   
1438042986022.41/warc/CC-MAIN-20150728002306-00...          Type   
1438042986022.41/warc/CC-MAIN-20150728002306-00...          Team   
1438042986022.41/warc/CC-MAIN-20150728002306-00...      Position   
1438042986022.41/warc/CC-MAIN-20150728002306-00...          Team   
1438042986022.41/warc/CC-MAIN-20150728002306-00...      Location   
1438042986022.41/warc/CC-MAIN-20150728002306-00...      Category   
1438042986022.41/warc/CC-MAIN-20150728002306-00...        Status   
1438042986022.41/warc/CC-MAIN-20150728002306-00...          Type   

                                                        header_c  n_values  \
table_id                                                                     
1438042986022.41/warc/CC-MAIN-20150728002306-00...  manufacturer         3   
1438042986022.41/warc/CC-MAIN-20150728002306-00...          year         4   
1438042986022.41/warc/CC-MAIN-20150728002306-00...          type       112   
1438042986022.41/warc/CC-MAIN-20150728002306-00...          team        25   
1438042986022.41/warc/CC-MAIN-20150728002306-00...      position        15   
1438042986022.41/warc/CC-MAIN-20150728002306-00...          team        25   
1438042986022.41/warc/CC-MAIN-20150728002306-00...      location         4   
1438042986022.41/warc/CC-MAIN-20150728002306-00...      category         4   
1438042986022.41/warc/CC-MAIN-20150728002306-00...        status         2   
1438042986022.41/warc/CC-MAIN-20150728002306-00...          type        50   

                                                    length-agg-any  \
table_id                                                             
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True   
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True   
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True   
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True   
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True   
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True   
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True   
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True   
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True   
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True   

                                                    length-agg-all  \
table_id                                                             
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True   
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True   
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True   
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True   
1438042986022.41/warc/CC-MAIN-20150728002306-00...           False   
1438042986022.41/warc/CC-MAIN-20150728002306-00...            True

In [7]:
import sys
import os
import json

def get_atd_dfs():
    os.chdir('../automatic-task-discovery')
    sys.path.append('../automatic-task-discovery')
    from src import load_metadata, process_csv_file_sato

    # get current working directory
    cwd = os.getcwd()
    # change to the directory of the script
    #os.chdir('../automatic-task-discovery')

    cwd = os.getcwd()
    read_path_csv="csv_data_full/"
    dict_metadata=load_metadata()
    for idx, (key, value) in enumerate(dict_metadata.items()):

        print(idx,key)
        # check that dataset is not eda only
        with open("data/labels/" + key + ".json") as f:
            dict_file=json.load(f)
        if dict_file["eda_only"]==1:
            continue
        
        folder_name=key
        df=process_csv_file_sato(read_path_csv, folder_name)

        result = {
            'df': df,
            'dataset_id': folder_name,
            'locator': "no locator"
        }
        yield result
        

In [10]:
df_iter=get_atd_dfs()
for i in range(2):
    dic=next(df_iter)
    print(dic['df'].head(2))
os.chdir('../sato')

['pierrelouisdanieau.french-department-temperature-since-2018.json', 'pavelbiz.4-228-398-unique-nicknames-of-minecraft-players.json', 'paytonfisher.sp-500-companies-with-financial-information.json', 'imakash3011.dogecoin-cryptocurrency-dataset.json', 'thedevastator.domain-analysis-of-5000-most-popular-github-repo.json', 'thedevastator.detailed-labelled-fishing-trajectories-from-ais.json', 'adnandodmani.world-timezone-dataset.json', 'amjaads.netflix-movies.json', 'asjad99.chicago-air-pollution.json', 'moezabid.american-bank-data.json', 'data40.facebook-cryptocurrency-and-nft-accounts-2022.json', 'jeffreybraun.chopped-10-years-of-episode-data.json', 'brittanycallahan.sc-beach-cleanups.json', 'atharvahpatil.brooklyn-99-dataset-season-14.json', 'sandhyakrishnan02.sannamarin-tweets.json', 'mathurinache.ethereum-tweets.json', 'rishidamarla.concussions-in-the-nfl-20122014.json', 'prasertk.top-200-passwords-by-country-2021.json', 'maksymshkliarevskyi.reddit-data-science-posts.json', 'brendan45

In [12]:
os.getcwd()

'/home/jonathangehmayr/sato'

In [9]:
sys.path

['/home/jonathangehmayr/sato',
 '/usr/lib/python39.zip',
 '/usr/lib/python3.9',
 '/usr/lib/python3.9/lib-dynload',
 '',
 '/home/jonathangehmayr/sato/.venv9/lib/python3.9/site-packages',
 '../automatic-task-discovery',
 '../sato',
 '../automatic-task-discovery',
 '../sato',
 '../sato',
 '../sato',
 '../sato']

In [2]:
import re

In [5]:
re.split('\W+', 'mosquito_Indicator avg_temp')

['mosquito_Indicator', 'avg_temp']

In [53]:
from string import punctuation

def split_string_by_uppercase_and_special_characters(string):
    substrings = []
    current_substring = ''

    for char in string:
        if char.isupper() and current_substring == '':
            current_substring += char
        elif char.islower() and current_substring == '':
            current_substring += char
        elif char in punctuation + ' ' and current_substring == '':
            continue
        elif char.isupper() and current_substring.isupper():
            current_substring += char
        elif current_substring[0].isupper() and len(current_substring)==1 and char.islower():
            current_substring += char
        elif char.islower() and current_substring.isupper():
            substrings.append(current_substring)
            current_substring = char
        elif char.islower() and current_substring.islower():
            current_substring += char
        elif char.isupper() and current_substring.islower():
            substrings.append(current_substring)
            current_substring = char
        elif char.islower():
            current_substring += char
        elif char.isupper():
            substrings.append(current_substring)
            current_substring = char
        elif char in punctuation + ' ':
            substrings.append(current_substring)
            current_substring = ''
        else:
            current_substring += char

    if current_substring:
        substrings.append(current_substring)
    
    substrings=[s.lower() for s in substrings]

    return substrings

# Example usage:
input_string = "HelloWorldHowAreYouToday-andTomorrow_Or yesterday_and_next_WeekYESSSS###"
#input_string ="HelloWorldHOWAREYOUtoday"
result = split_string_by_uppercase_and_special_characters(input_string)
print(result)

['hello', 'world', 'how', 'are', 'you', 'today', 'and', 'tomorrow', 'or', 'yesterday', 'and', 'next', 'week', 'yessss']


In [29]:
[*s.split('_') for s in result]

SyntaxError: iterable unpacking cannot be used in comprehension (1378111678.py, line 1)

In [43]:
type(punctuation)

str

In [45]:
punctuation + ' '

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ '

In [41]:
specials=list(punctuation).append(' ')
tyspecials)

NoneType

In [33]:
specials=""
for char in set(punctuation):
    specials.join(char)
specials

''

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~'}